## Request address index via address: pochta.ru/post-index

In [1]:
import pandas as pd
import numpy as np
import time
import re

#import socket
#from tqdm import tqdm
from random import choice, uniform
import requests
import json
from fake_useragent import UserAgent
import bs4

from multiprocessing import Pool, cpu_count
from multiprocessing.dummy import Pool as ThreadPool

from sqlalchemy import create_engine
import cx_Oracle #before import, execute from Anaconda prompt: conda install cx_Oracle

from IPython.core.display import display, HTML, display_html
display(HTML("<style>.container { width:99% !important; }</style>")) # set full window wide cells (only in this notebook)

# Display some dataframes in one cell from left to right 
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)
    
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 220)
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', '{:,.2f}'.format)

In [2]:
url = 'https://www.pochta.ru/portal-portlet/delegate/postoffice-api/method/offices.find.forAddress'
headers = {'Accept': 'application/json',}
cookies = {
    'JSESSIONID': '68ADFA92B4F5B1E7643D938E9C1952FE',
    'ANALYTICS_UUID': 'efda9147-5b7b-442d-82e2-9519f143ff49',
    'GUEST_LANGUAGE_ID': 'ru_RU',
    'COOKIE_SUPPORT': 'true',
    '_ga': 'GA1.2.883162138.1541063535',
    '_gid': 'GA1.2.128994641.1541063535',
    '_ym_uid': '1541063535465768396', 
    '_ym_d': '1541063535', 
    '_ym_isad': '2',
    'HeaderBusinessTooltip': 'showed',
    'webchat-uuid': 'c645b2a8-46c7-4783-9377-ccfeb584edad', 
    'sputnik_session': '1541072375545|0',
}

In [4]:
#PROXY IP (16k)
df_proxys = pd.read_excel('//home/data/parsing/proxy.xlsx')
df_proxys.head()

In [5]:
#TEST
random_proxy_idx = choice(df_proxys['proxy'].index)
df_proxys.loc[random_proxy_idx]['proxy']

'37.59.248.191:8080'

In [ ]:
df = pd.read_excel('Список адресов для получения индекса с pochta_ru (4463 ФЛ 2019-04-15).xlsx', dtype='object', index_col=0)
df = df.fillna('').reset_index(drop=True)
df = df.astype(str)
df['Индекс1'] = '0'
df['Индекс2'] = '0'
df = df.iloc[:,:15].join([df.iloc[:,-2:], df.iloc[:,15:-2]])
df['Полный адрес'] = df['Регион']+', '+df['Город']+', '+df['Поселение']+', '+df['Населенный пункт1']+', '+df['Улица1']+', '+df['Дом1']+', '+df['Корпус1']
df['Полный адрес'] = df['Полный адрес'].apply(lambda x: re.sub('(, )+', ', ', x))

#df['Регион'][df['Полный адрес'].str.contains('осковская обл')] = 'Московская область'

df['Адрес'] = df['Регион']+', '+df['Город']+', '+df['Поселение']+', '+df['Населенный пункт1']+', '+df['Улица1']+', '+df['Дом1']+', '+df['Корпус1']
df['Адрес'] = df['Адрес'].apply(lambda x: re.sub('(, )+', ', ', x))
df['Адрес_сокр'] = df['Регион']+', '+df['Город']+', '+df['Поселение']+', '+df['Населенный пункт1']
df['Адрес_сокр'] = df['Адрес_сокр'].apply(lambda x: re.sub('(, )+', ', ', x))
display_side_by_side(df.head(2), df.tail(2))

In [ ]:
post_counter = 0
global post_counter

In [46]:
def pochta_index(addr, use_proxy=True):
    data = {'address': addr}
    
    if use_proxy: #With proxy
        
        timeout=True
        while timeout:
            
            random_proxy_idx = choice(df_proxys['proxy'].index)
            while df_proxys.loc[random_proxy_idx,'retry'] > 0:
                random_proxy_idx = choice(df_proxys['proxy'].index)
                
            proxy = {'https': 'http://' + df_proxys.loc[random_proxy_idx]['proxy']}
            
            try:
                r = requests.post(url, headers=headers, cookies=cookies, data=data, proxies=proxy, timeout=2)
                timeout=False
                df_proxys.loc[random_proxy_idx,'retry'] -= 1
            except Exception as e:
                #print(e, 'Change proxy...', proxy)
                #print('Change proxy...', proxy)
                df_proxys.loc[random_proxy_idx,'retry'] += 1
                timeout=True
    else: #Without proxy
        r = requests.post(url, headers=headers, cookies=cookies, data=data)
    
    valid_json_string = "[" + r.text + "]"  # or "[{0}]".format(your_string)
    try:
        valid_json_string = json.loads(valid_json_string)
    except:
        valid_json_string = "[" + r.text + "]"
    return valid_json_string

In [47]:
def gis2_index(address):
    headers = {'userAgent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36',
           'BrowserBaseVersion': '72.0.3626.121', 'OSFamily': 'Windows', 'BrowserVersion': '72.0.3626.121', 
           'x64': 'true', 'OSVersion': '6.1', 'OSName': 'Windows 7', 'appCodeName': 'Mozilla', 'appName': 'Netscape', 
           'appVersion': '5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36',
           'doNotTrack': '1', 'platform': 'Win32', 'product': 'Gecko', 'productSub': '20030107', 'vendor': 'Google Inc.', 
           'geolocation': 'denied', 'notifications': 'denied'}

    url='https://beta.2gis.ru/moscow/search/'+ address +'?m=37.607557%2C55.790193%2F10.73'
    content = requests.get(url, headers=headers)
    soap = bs4.BeautifulSoup(content.text, "html.parser")
    try:
        url = soap.select('div._1h3cgic')[0].select('a')[0]['href']
    except:
        return '0'
    url_main = 'https://beta.2gis.ru'
    content = requests.get(url_main + url, headers=headers)
    soap = bs4.BeautifulSoup(content.text, "html.parser")
    try:
        index_org = soap.select('li._1w9o2igt')[1].text
    except:
        index_org = '0'
    return index_org

In [73]:
#TEST FOR ONE ADDRESS
pochta_index('г. Москва, ст. Мачихино, с/т "Полесье"')[0]['postOffices'][0]['index']

'108830'

In [52]:
def pochta_mp(df_sub):
    
    global post_counter
    addr_index1 = []
    addr_index2 = []
    addr = []
    for idx, i, j, k, l in zip(df_sub['Полный адрес'].index, df_sub['Полный адрес'], df_sub['Адрес'], df_sub['Адрес_сокр'], df['Регион']+', '+df['Поселение']):
        
        post_counter += 1
        print(idx, end=' ')
        if post_counter % 100 == 0:
            print('\n',time.ctime()[11:-5], 'Counter:', post_counter)
            
        content1 = pochta_index(i)
        try:
            content1 = content1[0]['postOffices'][0]['index']
        except:
            
            content1 = pochta_index(j)
            try:
                content1 = content1[0]['postOffices'][0]['index']
            except:
        
                content1 = pochta_index(k)
                try:
                    content1 = content1[0]['postOffices'][0]['index']
                except:
        
                    content1 = pochta_index(l)
                    try:
                        content1 = content1[0]['postOffices'][0]['index']
                    except:
                        content1 = '0'

        content2 = gis2_index(i)
        try:
            content2 = int(content2)
        except:
            
            content2 = gis2_index(j)
            try:
                content2 = int(content2)
            except:
        
                content2 = gis2_index(k)
                try:
                    content2 = int(content2)
                except:

                    content2 = gis2_index(l)
                    try:
                        content2 = int(content2)
                    except:
                        content2 = '0'

        addr_index1.append(content1)
        addr_index2.append(content2)
        addr.append(i)
        
        
    return pd.DataFrame({'addr_index1': addr_index1, 'addr_index2': addr_index2, 'addr': addr})

In [ ]:
#First try
post_counter = 0
print('Indexes for post:', len(df[df['Индекс']=='']))

print(time.ctime()[11:-5], 'Start')


try_num = 1
while ( len(df[df['Индекс']==''])>0 ) and ( try_num<=3 ):

    index_for_check = df[df['Индекс']==''].index

    try: # Delete MP pool, if it present!
        pool.close()
        print('old pool deleted!')
    except NameError:
        print('pool absent!')

    try: # try to use MP, get proc count
        cpus = cpu_count()
    except NotImplementedError:
        cpus = 2   # arbitrary default

    if len(index_for_check)<16:
        cpus = len(index_for_check)
    #cpus=1 # Set lower proc count, if memory not enough
    print('Use', cpus, 'threads')

    pool = ThreadPool(processes=cpus)

    df_index = pd.concat( pool.map( pochta_mp, pd.np.array_split(df.loc[index_for_check], cpus) ), ignore_index=True)#.reset_index(drop=True)
    
    #Close pool
    pool.close()
    pool.join()
    
    df['Индекс1'].loc[index_for_check] = np.array(df_index['addr_index1'])
    df['Индекс2'].loc[index_for_check] = np.array(df_index['addr_index2'])
    #display_side_by_side(df.loc[index_for_check])
    print('Not found in Pochta:', len(df[df['Индекс1']=='0']))
    print('Not found in 2Gis:', len(df[df['Индекс2']=='0']))
    try_num += 1


print(time.ctime()[11:-5], 'Finish')
#display_side_by_side(df_index[df_index['addr_index']=='0'])
df

In [54]:
df_all = df_index.copy()
df_all.to_excel('Список индексов с pochta_ru (2019-04-15 ФЛ).xlsx')

In [ ]:








#TEST AND SAVE PROXY GATES WORK STATUS

In [11]:
#proxy status level: negative value - good worked proxy
len(df_proxys[df_proxys['retry']>0]), len(df_proxys[df_proxys['retry']<0])

(20761, 722)

In [12]:
df_proxys['retry'].value_counts()

 1     20725
-4        58
-6        53
-5        51
-9        49
-11       45
-7        43
-10       42
-2        37
-8        37
 2        36
-12       36
-1        36
-3        33
-14       32
-16       30
-13       24
-15       23
 0        22
-17       21
-19       14
-18       13
-20       10
-21        8
-24        5
-25        5
-26        4
-22        4
-23        3
-27        2
-28        2
-30        1
-31        1
Name: retry, dtype: int64

In [13]:
df_proxys.to_excel('//home/data/parsing/proxy.xlsx')